## Project Title: To determine if data augmentation using the method proposed in 'Finding Order in Chaos: A Novel Data Augmentation Method for Time Series in Contrastive Learning' will lead to better 1 day prediction results.

In [43]:
import tensorflow as tf
import pandas as pd
import yfinance as yf
import seaborn as sb
from tensorflow.keras import layers, Model
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import rfft, rfftfreq, irfft
from IPython.display import display, HTML
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tcn import TCN  # If you have the tcn package installed
from sklearn.metrics import mean_squared_error

display(HTML("<style>.container { width:100% !important; }</style>"))

In [42]:
# Function to import stock data
def get_stock_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    return data['Close']

In [3]:
# Function to perform Fourier Transform and plot
def fourier_transform_and_plot(stock_data, title):
    # Apply the Fourier Transform to the closing price data
    yf = rfft(stock_data)
    xf = rfftfreq(len(stock_data), 1)  # Assuming daily samples for frequency calculation
    
    # Plot the Fourier Transform results
    plt.figure(figsize=(14, 7))
    plt.plot(xf, np.abs(yf))  # Plot the magnitude spectrum
    plt.title(f'Fourier Transform of {title}')
    plt.xlabel('Frequency (1/day)')
    plt.ylabel('Amplitude')
    plt.grid()
    plt.show()

    # Return the frequency and magnitude for further analysis if needed
    return xf, np.abs(yf)

### Extract data

In [4]:
ticker1 = 'AAPL'
ticker2 = 'MSFT'
start_date = '2010-01-01'
end_date = '2023-01-01'

# Import data
close1 = get_stock_data(ticker1, start_date, end_date)
close2 = get_stock_data(ticker2, start_date, end_date)

### Next steps
1. Idenitify sinusoidal components with Fourier Transform
2. Calculate Phase Difference for mixup coefficient for each significant coefficient
3. Reconstruct time series using inverse Fourier Transform

In [5]:
def get_significant_frequencies(data, threshold=0.1):
    """
    Perform Fourier Transform on data and identify frequencies with significant amplitude.
    
    Args:
    - data: Time series data.
    - threshold: Threshold for significance, relative to the max amplitude.
    
    Returns:
    - significant_freq: Frequencies with significant amplitude.
    - significant_ampl: Amplitude of the significant frequencies.
    - full_spectrum: Full Fourier spectrum for all frequencies.
    """
    # Perform Fourier Transform
    spectrum = rfft(data)
    frequencies = rfftfreq(data.size, d=1)  # Assuming unit time interval between samples
    
    # Find significant amplitudes
    amplitude = np.abs(spectrum)
    significant_indices = amplitude > (amplitude.max() * threshold)
    significant_freq = frequencies[significant_indices]
    significant_ampl = amplitude[significant_indices]
    
    return significant_freq, significant_ampl, spectrum

# def phase_mixup(sig_freq1, sig_ampl1, spectrum1, sig_freq2, sig_ampl2, spectrum2, alpha=0.5):
#     """
#     Combine two signals based on phase and magnitude mixup technique.
    
#     Args:
#     - sig_freq1: Significant frequencies of the first signal.
#     - sig_ampl1: Significant amplitudes of the first signal.
#     - spectrum1: Full Fourier spectrum of the first signal.
#     - sig_freq2: Significant frequencies of the second signal.
#     - sig_ampl2: Significant amplitudes of the second signal.
#     - spectrum2: Full Fourier spectrum of the second signal.
#     - alpha: Mixup coefficient (0 to 1).
    
#     Returns:
#     - mixed_spectrum: The mixed spectrum after applying mixup.
#     """
#     # Make a copy of the first spectrum to start with
#     mixed_spectrum = np.copy(spectrum1)
#     freqs = rfftfreq(spectrum1.size, d=1)
    
#     # Calculate mixup for each significant frequency
#     for freq in sig_freq1:
#         # Find the index of the nearest frequency
#         index = np.argmin(np.abs(freqs - freq))
#         phase1 = np.angle(spectrum1[index])
#         phase2 = np.angle(spectrum2[index])
        
#         # Calculate shortest phase difference
#         phase_diff = (phase2 - phase1) % (2 * np.pi)
#         phase_diff = phase_diff - 2 * np.pi if phase_diff > np.pi else phase_diff
        
#         # Calculate new amplitude and phase
#         new_amplitude = alpha * sig_ampl1[index] + (1 - alpha) * sig_ampl2[index]
#         new_phase = phase1 + alpha * phase_diff
        
#         # Apply mixup
#         mixed_spectrum[index] = new_amplitude * np.exp(1j * new_phase)
    
#     return mixed_spectrum

def phase_mixup(sig_freq1, sig_ampl1, spectrum1, sig_freq2, sig_ampl2, spectrum2, alpha=0.5):
    mixed_spectrum = np.copy(spectrum1)
    freqs1 = rfftfreq(spectrum1.size, d=1)
    freqs2 = rfftfreq(spectrum2.size, d=1)
    
    for freq in sig_freq1:
        index1 = np.argmin(np.abs(freqs1 - freq))
        index2 = np.argmin(np.abs(freqs2 - freq))
        
        if index1 >= len(sig_ampl1) or index2 >= len(sig_ampl2):
            continue  # Skip the frequency if the index is out of bounds

        phase1 = np.angle(spectrum1[index1])
        phase2 = np.angle(spectrum2[index2])
        
        phase_diff = (phase2 - phase1) % (2 * np.pi)
        phase_diff = phase_diff - 2 * np.pi if phase_diff > np.pi else phase_diff
        
        new_amplitude = alpha * sig_ampl1[index1] + (1 - alpha) * sig_ampl2[index2]
        new_phase = phase1 + alpha * phase_diff
        
        mixed_spectrum[index1] = new_amplitude * np.exp(1j * new_phase)
    
    return mixed_spectrum


def reconstruct_time_series(mixed_spectrum):
    """
    Reconstruct time series from mixed spectrum using inverse Fourier Transform.
    
    Returns:
    - mixed_time_series: The reconstructed time series.
    """
    # Perform inverse Fourier Transform
    mixed_time_series = irfft(mixed_spectrum)
    
    return mixed_time_series

In [6]:
# Step 1: Get significant frequencies and amplitude for both time series
sig_freq1, sig_ampl1, spectrum1 = get_significant_frequencies(close1.values)
sig_freq2, sig_ampl2, spectrum2 = get_significant_frequencies(close2.values)

# Step 2: Identify significant frequencies (already done in step 1)

# Step 3: Phase and Magnitude Mixup
mixed_spectrum = phase_mixup(sig_freq1, sig_ampl1, spectrum1, sig_freq2, sig_ampl2, spectrum2)

# Step 4: Reconstruction of the time series
mixed_time_series = reconstruct_time_series(mixed_spectrum)

mixed_time_series[:10]  # Display the first 10 points of the mixed time series

In [7]:
def augment_data(ts1, ts2):
    # Step 1: Get significant frequencies and amplitude for both time series
    sig_freq1, sig_ampl1, spectrum1 = get_significant_frequencies(ts1.values)
    sig_freq2, sig_ampl2, spectrum2 = get_significant_frequencies(ts2.values)

    # Step 2: Identify significant frequencies (already done in step 1)

    # Step 3: Phase and Magnitude Mixup
    mixed_spectrum = phase_mixup(sig_freq1, sig_ampl1, spectrum1, sig_freq2, sig_ampl2, spectrum2)

    # Step 4: Reconstruction of the time series
    mixed_time_series = reconstruct_time_series(mixed_spectrum)
    
    return mixed_time_series

In [8]:
# Plot the mixed time series and the original ones on the same plot
plt.figure(figsize=(14, 7))  # Set the figure size for better visibility
plt.plot(mixed_time_series, label='Mixed Time Series')
plt.plot(close1.values, label='AAPL')
plt.plot(close2.values, label='MSFT')
plt.title('Mixed Time Series and Original Time Series')
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.legend()  # Show the legend to identify each line
plt.tight_layout()
plt.show()


### Gather all 30 stock data from DJIA

In [9]:
# Define the list of Dow Jones Industrial Average companies
tickers = [
    "MMM", "AXP", "AMGN", "AAPL", "BA", "CAT", "CVX", "CSCO", "KO", "DIS",
    "DOW", "GS", "HD", "HON", "IBM", "INTC", "JNJ", "JPM", "MCD", "MRK",
    "MSFT", "NKE", "PG", "CRM", "TRV", "UNH", "V", "WBA", "WMT"
]

# Create a dictionary to store historical data for each company
historical_data = {}

# Loop through the Dow companies and retrieve historical data
for ticker in tickers:
    stock_data = get_stock_data(ticker, start_date, end_date)
    historical_data[ticker] = stock_data

In [10]:
historical_data_df = pd.DataFrame(historical_data)

#### Question to ask myself.
Let N be the number of new datasets and Let C be the set of N stocks in DJIA that are selected to be paired with the selected stock. i.e. if I want to predict the next day stock return of AAPL, which 10/15/20 stocks should I augment AAPL with to create new features out of them.

How do I choose N?
- Iteratively and optimise based on MSE/MAE/Sharpe
- Run statistical tests such as paired t-tests to compare model's performance with different volumes of augmented data against the baseline of real data only (find the inflexion point)

How do I select the N stocks?
- correlation
- sectors
- clustering with the use of unsupervised learning techniques
- random

## Determine the number of augmented datasets to use
In this case, we will use AAPL as our control factor.

### Base case
Using only real AAPL data

sidenote: Train test split will be a 2/3, 1/3 split

In [11]:
aapl = historical_data['AAPL']

In [12]:
aapl

In [13]:
def create_features(stock_data, ticker, windows=[10, 20, 40, 60, 80, 120, 240]):
    """
    Create time series features for stock data including rolling volatility,
    normalized simple moving averages, rolling median to mean ratio,
    rolling standard deviation to mean ratio, and rolling return.

    Args:
    stock_data (pd.DataFrame): DataFrame with a 'Close' column containing stock prices.
    windows (list of int): List of window sizes for calculating rolling features.

    Returns:
    pd.DataFrame: DataFrame with new features.
    """
    stock_data = pd.DataFrame(stock_data)

    for window in windows:
        # Rolling Volatility (standard deviation of returns)
        stock_data[f'Volatility_{window}d_{ticker}'] = stock_data['Close'].pct_change().rolling(window=window).std()

        # Simple Moving Average normalized by 5-day Simple Moving Average
        sma = stock_data['Close'].rolling(window=window).mean()
        sma_5d = stock_data['Close'].rolling(window=5).mean()
        stock_data[f'SMA_{window}d_norm_{ticker}'] = sma / sma_5d

        # Rolling Median to Mean Ratio
        median = stock_data['Close'].rolling(window=window).median()
        mean = stock_data['Close'].rolling(window=window).mean()
        stock_data[f'Median_to_Mean_{window}d_{ticker}'] = median / mean

        # Rolling Standard Deviation to Mean Ratio
        std = stock_data['Close'].rolling(window=window).std()
        stock_data[f'Std_to_Mean_{window}d_{ticker}'] = std / mean

    # Daily returns
    stock_data[f'Returns_{ticker}'] = stock_data['Close'].pct_change() * 100

    # Drop the initial rows with NaN values due to rolling windows
    stock_data = stock_data.dropna()
    
    y = stock_data[f"Returns_{ticker}"]
    x = stock_data.drop(columns=['Close', f'Returns_{ticker}'])

    return x, y

In [14]:
x, y = create_features(aapl,'MSFT')
print(x.shape, y.shape)

# Train Test Split
x_train = x[:int(x.shape[0]*2/3)]
x_test = x[int(x.shape[0]*2/3):]

y_train = y[:int(y.shape[0]*2/3)]
y_test = y[int(y.shape[0]*2/3):]

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

In [15]:
x_train

### Fit a XGBoost model

In [16]:
import xgboost as xgb

In [17]:
# Base params
params = {'max_depth': 2,
 'min_child_weight': 2,
 'gamma': 1,
 'subsample': 1.0,
 'colsample_bytree': 1.0,
 'reg_lambda': 1,
 'reg_alpha': 0,
 'learning_rate': 0.01,
 'n_estimators': 1000
}

matrix_train = xgb.DMatrix(x_train,label=y_train)
matrix_test = xgb.DMatrix(x_test,label=y_test)

evals = [(matrix_train, 'train'), (matrix_test, 'test')]

model = xgb.XGBRegressor(**params)
model.fit(x_train, y_train)

preds_train = model.predict(x_train)
preds_test = model.predict(x_test)

In [18]:
# Get feature importances
feature_importances = model.feature_importances_[2:]

# Get feature names
feature_names = x_train.columns.tolist()[2:]

# Sort the feature importances in descending order and match them with their names
indices = np.argsort(feature_importances)[::-1]
sorted_feature_names = [feature_names[i] for i in indices]

plt.figure(figsize=(10, 6))
plt.title("Feature Importances in XGBoost")
plt.barh(range(len(feature_importances)), feature_importances[indices], align='center', color='skyblue')
plt.yticks(range(len(feature_importances)), sorted_feature_names)
plt.gca().invert_yaxis()  # Invert y-axis to have the most important feature on top
plt.xlabel("Feature Importance")
plt.ylabel("Feature")
plt.tight_layout()
plt.show()

In [19]:
# calculating RMSE
rmse_train = np.sqrt(np.mean((preds_train-y_train.values)**2))
rmse_test = np.sqrt(np.mean((preds_test-y_test.values)**2))
print(f'The RMSE for train is: {round(rmse_train,3)} and test is {round(rmse_test,3)}.')

### Data Augmentation

In [20]:
# 1. N = 5 using top correlated pairs
returns_df = pd.DataFrame(historical_data).pct_change().iloc[1:]
returns_df

In [21]:
# Calculate the correlation matrix of the returns
correlation_matrix = returns_df.corr()
aapl_corr = correlation_matrix['AAPL'].sort_values(ascending=False)
top_5_pairs = list(aapl_corr[1:6].index)

top_5_pairs

In [22]:
aapl_corr = correlation_matrix['AAPL'].sort_values(ascending=True)
bot_5_pairs = list(aapl_corr[:5].index)

bot_5_pairs

In [78]:
# Control is AAPL
aapl = historical_data_df['AAPL']
msft = historical_data_df['MSFT']

# Create the augmented data for N = 1 
aug_ts1 = augment_data(aapl, msft)
aug_ts1 = pd.Series(aug_ts1, index=aapl.index, name='Close')

# Create features
x, y = create_features(aug_ts1, 'MSFT')

# Train Test Split
x_train = x[:int(x.shape[0]*2/3)]
x_test = x[int(x.shape[0]*2/3):]

y_train = y[:int(y.shape[0]*2/3)]
y_test = y[int(y.shape[0]*2/3):]

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

In [25]:
# Base params
params = {'max_depth': 2,
 'min_child_weight': 2,
 'gamma': 1,
 'subsample': 1.0,
 'colsample_bytree': 1.0,
 'reg_lambda': 1,
 'reg_alpha': 0,
 'learning_rate': 0.01,
 'n_estimators': 1000
}

matrix_train = xgb.DMatrix(x_train,label=y_train)
matrix_test = xgb.DMatrix(x_test,label=y_test)

evals = [(matrix_train, 'train'), (matrix_test, 'test')]

model = xgb.XGBRegressor(**params)
model.fit(x_train, y_train)

preds_train = model.predict(x_train)
preds_test = model.predict(x_test)

In [27]:
# calculating RMSE
rmse_train = np.sqrt(np.mean((preds_train-y_train.values)**2))
rmse_test = np.sqrt(np.mean((preds_test-y_test.values)**2))
print(f'The RMSE for train is: {round(rmse_train,3)} and test is {round(rmse_test,3)}.')

#### Mini experiment using the least correlated ts

In [29]:
# Control is AAPL
aapl = historical_data_df['AAPL']
mrk = historical_data_df['WMT']

# Create the augmented data for N = 1 
aug_ts1 = augment_data(aapl, mrk)
aug_ts1 = pd.Series(aug_ts1, index=aapl.index, name='Close')

# Create features
x, y = create_features(aug_ts1, 'WMT')

# Train Test Split
x_train = x[:int(x.shape[0]*2/3)]
x_test = x[int(x.shape[0]*2/3):]

y_train = y[:int(y.shape[0]*2/3)]
y_test = y[int(y.shape[0]*2/3):]

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

In [30]:
# Base params
params = {'max_depth': 2,
 'min_child_weight': 2,
 'gamma': 1,
 'subsample': 1.0,
 'colsample_bytree': 1.0,
 'reg_lambda': 1,
 'reg_alpha': 0,
 'learning_rate': 0.01,
 'n_estimators': 1000
}

matrix_train = xgb.DMatrix(x_train,label=y_train)
matrix_test = xgb.DMatrix(x_test,label=y_test)

evals = [(matrix_train, 'train'), (matrix_test, 'test')]

model = xgb.XGBRegressor(**params)
model.fit(x_train, y_train)

preds_train = model.predict(x_train)
preds_test = model.predict(x_test)

In [31]:
# Get feature importances
feature_importances = model.feature_importances_[2:]

# Get feature names
feature_names = x_train.columns.tolist()[2:]

# Sort the feature importances in descending order and match them with their names
indices = np.argsort(feature_importances)[::-1]
sorted_feature_names = [feature_names[i] for i in indices]

plt.figure(figsize=(10, 6))
plt.title("Feature Importances in XGBoost")
plt.barh(range(len(feature_importances)), feature_importances[indices], align='center', color='skyblue')
plt.yticks(range(len(feature_importances)), sorted_feature_names)
plt.gca().invert_yaxis()  # Invert y-axis to have the most important feature on top
plt.xlabel("Feature Importance")
plt.ylabel("Feature")
plt.tight_layout()
plt.show()

In [32]:
# calculating RMSE
rmse_train = np.sqrt(np.mean((preds_train-y_train.values)**2))
rmse_test = np.sqrt(np.mean((preds_test-y_test.values)**2))
print(f'The RMSE for train is: {round(rmse_train,3)} and test is {round(rmse_test,3)}.')

Observations. We can see that using a ts of a lower correlation can lead to greater overfit. Overall still a better fit than using the original data though.


#### N = 5 (for fun)

In [33]:
top_5_pairs

In [34]:
df_x, df_y = pd.DataFrame(), pd.DataFrame()
for ticker in top_5_pairs:
    ts = historical_data_df[ticker]
    aug_ts = augment_data(aapl, ts)
    aug_ts1 = pd.Series(aug_ts1, index=aapl.index, name='Close')
    
    # Create features
    x, y = create_features(aug_ts1, ticker)
    df_x = pd.concat([df_x, x], axis = 1)
    df_y = pd.concat([df_y, y], axis = 1)    

In [55]:
# Train Test Split
x_train = df_x[:int(df_x.shape[0]*2/3)]
x_test = df_x[int(df_x.shape[0]*2/3):]

y_train = df_y[:int(df_y.shape[0]*2/3)]
y_test = df_y[int(df_y.shape[0]*2/3):]

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

In [36]:
# Base params
params = {'max_depth': 2,
 'min_child_weight': 2,
 'gamma': 1,
 'subsample': 1.0,
 'colsample_bytree': 1.0,
 'reg_lambda': 1,
 'reg_alpha': 0,
 'learning_rate': 0.01,
 'n_estimators': 1000
}

matrix_train = xgb.DMatrix(x_train,label=y_train)
matrix_test = xgb.DMatrix(x_test,label=y_test)

evals = [(matrix_train, 'train'), (matrix_test, 'test')]

model = xgb.XGBRegressor(**params)
model.fit(x_train, y_train)

preds_train = model.predict(x_train)
preds_test = model.predict(x_test)

In [37]:
# Get feature importances
feature_importances = model.feature_importances_[2:]

# Get feature names
feature_names = x_train.columns.tolist()[2:]

# Sort the feature importances in descending order and match them with their names
indices = np.argsort(feature_importances)[::-1]
sorted_feature_names = [feature_names[i] for i in indices]

plt.figure(figsize=(10, 6))
plt.title("Feature Importances in XGBoost")
plt.barh(range(len(feature_importances)), feature_importances[indices], align='center', color='skyblue')
plt.yticks(range(len(feature_importances)), sorted_feature_names)
plt.gca().invert_yaxis()  # Invert y-axis to have the most important feature on top
plt.xlabel("Feature Importance")
plt.ylabel("Feature")
plt.tight_layout()
plt.show()

In [38]:
# calculating RMSE
rmse_train = np.sqrt(np.mean((preds_train-y_train.values)**2))
rmse_test = np.sqrt(np.mean((preds_test-y_test.values)**2))
print(f'The RMSE for train is: {round(rmse_train,3)} and test is {round(rmse_test,3)}.')

Observation: Clearly can see there is an overfit and too many features were being created.

### Add 3 other models into play
1. LSTM
2. GRU
3. TCN

In [82]:
# Control is AAPL
aapl = historical_data_df['AAPL']
msft = historical_data_df['MSFT']

# Create the augmented data for N = 1 
aug_ts1 = augment_data(aapl, mrk)
aug_ts1 = pd.Series(aug_ts1, index=aapl.index, name='Close')

# Create features
x, y = create_features(aug_ts1, 'MSFT')

# Train Test Split
x_train = x[:int(x.shape[0]*2/3)]
x_test = x[int(x.shape[0]*2/3):]

y_train = y[:int(y.shape[0]*2/3)]
y_test = y[int(y.shape[0]*2/3):]

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

In [94]:
# Convert DataFrame to correct shape for LSTM, GRU, and TCN
# Assuming each sample is one time step and each feature is one column in the DataFrame
x_train_reshaped = x_train.values.reshape((x_train.shape[0], 1, x_train.shape[1]))
x_test_reshaped = x_test.values.reshape((x_test.shape[0], 1, x_test.shape[1]))

# Convert target arrays to 1D numpy arrays for regression output
y_train_reshaped = y_train.values.flatten()
y_test_reshaped = y_test.values.flatten()

# Define early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


# Define a function to create and train an LSTM model
def create_train_lstm(x_train, y_train, x_test, y_test):
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(x_train, y_train, epochs=100, batch_size=32, validation_data=(x_test, y_test), callbacks=[early_stopping], verbose=0)
    return model

# Define a function to create and train a GRU model
def create_train_gru(x_train, y_train, x_test, y_test):
    model = Sequential()
    model.add(GRU(50, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(x_train, y_train, epochs=100, batch_size=32, validation_data=(x_test, y_test), callbacks=[early_stopping], verbose=0)
    return model

# Define a function to create and train a TCN model
def create_train_tcn(x_train, y_train, x_test, y_test):
    model = Sequential()
    model.add(TCN(50, input_shape=(x_train.shape[1], x_train.shape[2])))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(x_train, y_train, epochs=100, batch_size=32, validation_data=(x_test, y_test), callbacks=[early_stopping], verbose=0)
    return model

# Function to evaluate RMSE
def evaluate_rmse(model, x, y_true):
    y_pred = model.predict(x)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    return rmse

# Train and evaluate LSTM
lstm_model = create_train_lstm(x_train_reshaped, y_train_reshaped, x_test_reshaped, y_test_reshaped)
lstm_train_rmse = evaluate_rmse(lstm_model, x_train_reshaped, y_train_reshaped)
lstm_test_rmse = evaluate_rmse(lstm_model, x_test_reshaped, y_test_reshaped)

# Train and evaluate GRU
gru_model = create_train_gru(x_train_reshaped, y_train_reshaped, x_test_reshaped, y_test_reshaped)
gru_train_rmse = evaluate_rmse(gru_model, x_train_reshaped, y_train_reshaped)
gru_test_rmse = evaluate_rmse(gru_model, x_test_reshaped, y_test_reshaped)

# # Train and evaluate TCN
tcn_model = create_train_tcn(x_train_reshaped, y_train_reshaped, x_test_reshaped, y_test_reshaped)
tcn_train_rmse = evaluate_rmse(tcn_model, x_train_reshaped, y_train_reshaped)
tcn_test_rmse = evaluate_rmse(tcn_model, x_test_reshaped, y_test_reshaped)

# Compare RMSEs
print(f"LSTM Train RMSE: {round(lstm_train_rmse,4)}, Test RMSE: {round(lstm_test_rmse,4)}")
print(f"GRU Train RMSE: {round(gru_train_rmse,4)}, Test RMSE: {round(gru_test_rmse,4)}")
print(f"TCN Train RMSE: {round(tcn_train_rmse,4)}, Test RMSE: {round(tcn_test_rmse,4)}")

In [115]:
from tensorflow.keras.callbacks import EarlyStopping
from kerastuner import HyperModel, RandomSearch
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Define your hypermodel
class LSTMHyperModel(HyperModel):
    def __init__(self, input_shape):
        self.input_shape = input_shape

    def build(self, hp):
        model = Sequential()
        model.add(LSTM(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu', input_shape=self.input_shape))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mean_squared_error')
        return model

# Create a function that returns a new EarlyStopping callback for each trial
def build_callbacks():
    return [
        EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    ]

# Set up the tuner
tuner = RandomSearch(
    hypermodel=LSTMHyperModel(input_shape=(x_train_reshaped.shape[1], x_train_reshaped.shape[2])),
    objective='val_loss',
    max_trials=10,
    executions_per_trial=2,
    directory='my_dir',
    project_name='lstm_hyperopt'
)

# Perform the hyperparameter search
tuner.search(
    x_train_reshaped, y_train_reshaped,
    epochs=100,
    validation_split=0.2,
    callbacks=build_callbacks()
)

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Train the best model
best_model = tuner.hypermodel.build(best_hps)
best_model.fit(
    x_train_reshaped, y_train_reshaped,
    epochs=50,
    validation_split=0.2,
    callbacks=build_callbacks()
)


In [117]:
best_hps